<a href="https://colab.research.google.com/github/JacopoMangiavacchi/Swift-TensorFlow-Sample-Notebooks/blob/master/FizzBuzz_Swift_TensorFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import Foundation
import TensorFlow

In [0]:
func bitEncode(_ n: UInt, bits: Int = 10) -> [Float] {
    var array = [Float](repeating: 0, count: bits)
    for i in 0..<bits {
        array[bits - 1 - i] = Float(n >> i & 1)
    }
    return array
}

func labelEncode(_ n: UInt) -> [Float] {
    guard n > 0 else { return [0, 0, 0, 1] }
    var array: [Float] = [0, 0, 0, 0]

    if n % 15 == 0 {
        array[0] = 1
    }
    else if n % 3 == 0 {
        array[1] = 1
    }
    else if n % 5 == 0 {
        array[2] = 1
    }
    else {
        array[3] = 1
    }

    return array
}

In [0]:
let start = 1
let end = 100
var X = [[Float]]()
var Y = [[Float]]()

for i in start..<end {
    X.append(bitEncode(UInt(i)))
    Y.append(labelEncode(UInt(i)))
}

In [0]:
struct FizzBuzz: Layer {
    var l1: Dense<Float>
    var l2: Dense<Float>
    var l3: Dense<Float>


    init(bits: Int = 1) {
        l1 = Dense<Float>(inputSize: bits, outputSize: 64, activation: relu)
        l2 = Dense<Float>(inputSize: 64, outputSize: 128, activation: relu)
        l3 = Dense<Float>(inputSize: 128, outputSize: 4, activation: softmax) // {$0}
    }
    
    @differentiable(wrt: (self, input))
    func applied(to input: Tensor<Float>) -> Tensor<Float> {
        let output1 = l1.applied(to: input)
        let output2 = l2.applied(to: output1)
        return l3.applied(to: output2)

    }
}

In [5]:
let x = Tensor<Float>(shape: [Int32(end - start), 10], scalars: Array(X.joined()))
let y = Tensor<Float>(shape: [Int32(end - start), 4], scalars: Array(Y.joined()))

print(x.shape)
print(y.shape)

TensorShape(dimensions: [99, 10])
TensorShape(dimensions: [99, 4])


In [0]:
let batchSize: Int32 = 128
let trainingIterations = Int32(end - start) / batchSize

func minibatch<T>(_ x: Tensor<T>, batchIndex: Int32) -> Tensor<T> {
  let start = batchIndex * batchSize
  return x[start..<start+batchSize]
}

In [7]:
let optimizer = RMSProp<FizzBuzz, Float>(learningRate: 0.003)
var model = FizzBuzz(bits: 10)


for epoch in 1...10000 {
    var totalLoss: Float = 0

    for i in 0..<trainingIterations {
        let xBatch = minibatch(x, batchIndex: i)
        let yBatch = minibatch(y, batchIndex: i)

        let gradients = gradient(at: model) { m -> Tensor<Float> in
            let ŷ = m.applied(to: xBatch)
            let batchLoss = softmaxCrossEntropy(logits: ŷ, labels: yBatch)
            totalLoss += batchLoss.scalarized()
            return batchLoss
        }
        optimizer.update(&model.allDifferentiableVariables, along: gradients)
    }


    if epoch % 1000 == 0 {
        print("Epoch \(epoch) loss: \(totalLoss)")
    }
}

Epoch 1000 loss: 0.0
Epoch 2000 loss: 0.0
Epoch 3000 loss: 0.0
Epoch 4000 loss: 0.0
Epoch 5000 loss: 0.0
Epoch 6000 loss: 0.0
Epoch 7000 loss: 0.0
Epoch 8000 loss: 0.0
Epoch 9000 loss: 0.0
Epoch 10000 loss: 0.0


In [0]:
// let optimizer = RMSProp<FizzBuzz, Float>(learningRate: 0.003)
// var model = FizzBuzz(bits: 10)


// for epoch in 1...1000 {
//     let (cost, 𝛁model) = model.valueWithGradient { m -> Tensor<Float> in
//         let ŷ = m.applied(to: x)
//         return softmaxCrossEntropy(logits: ŷ, labels: y)
//     }
//     optimizer.update(&model.allDifferentiableVariables, along: 𝛁model)
  
//     if epoch % 10 == 0 {
//         print("Epoch: \(epoch) Cost: \(cost)")
//     }
// }

In [0]:
var X_test = [[Float]]()
var Y_true = [[Float]]()

for i in 0..<100 {
    X_test.append(bitEncode(UInt(i)))
    Y_true.append(labelEncode(UInt(i)))
}

let xTest = Tensor<Float>(shape: [100, 10], scalars: Array(X_test.joined()))
let yTest = model.applied(to: xTest)

In [10]:
let yMax = yTest.argmax(squeezingAxis: 1).array
let yArray = Array(yTest.array)
let description = ["FizzBuzz", "Fizz    ", "Buzz    ", "        "]

for i in 0..<100 {
    print(i, description[Int(yMax[i].description)!], Y_true[i], yArray[i])
}

0 FizzBuzz [0.0, 0.0, 0.0, 1.0] [0.25, 0.25, 0.25, 0.25]
1          [0.0, 0.0, 0.0, 1.0] [0.25509593, 0.23842248, 0.23629057, 0.270191]
2          [0.0, 0.0, 0.0, 1.0] [0.24213266, 0.25685388, 0.22209178, 0.27892157]
3          [0.0, 1.0, 0.0, 0.0] [0.2630091, 0.2397849, 0.21642081, 0.28078523]
4 Buzz     [0.0, 0.0, 0.0, 1.0] [0.24197285, 0.245017, 0.25743186, 0.25557828]
5          [0.0, 0.0, 1.0, 0.0] [0.25204873, 0.23316228, 0.2332494, 0.2815396]
6          [0.0, 1.0, 0.0, 0.0] [0.24434917, 0.25553596, 0.19659962, 0.30351523]
7          [0.0, 0.0, 0.0, 1.0] [0.25092223, 0.24028395, 0.19857872, 0.31021512]
8 Buzz     [0.0, 0.0, 0.0, 1.0] [0.2530235, 0.24376775, 0.25791526, 0.24529356]
9 FizzBuzz [0.0, 1.0, 0.0, 0.0] [0.26550248, 0.23208976, 0.24309665, 0.2593111]
10          [0.0, 0.0, 1.0, 0.0] [0.24269448, 0.22957502, 0.24677868, 0.28095192]
11          [0.0, 0.0, 0.0, 1.0] [0.27469966, 0.22789234, 0.21440391, 0.28300408]
12 Buzz     [0.0, 1.0, 0.0, 0.0] [0.25066748, 0.22969352, 0.